# Examen Final Lucas Ribas

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [1]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [2]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [3]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [4]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [5]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [6]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [7]:
gamma = 1
sigma = .005

In [8]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [9]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

In [10]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .5504 .


Pregunta 2:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,110.00 (365 días después).


Pregunta 3:

Considere un swap a 1Y con cupones semestrales de 1.2% (lineal). Si la tasa
cupón cero a 0.5Y es 1.1% (exp), calcule la tasa cero a 1Y.


Pregunta 4:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 5:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 99.0%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 6:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tas

## Respuestas

### Pregunta 1

In [37]:
r0 = curva(0.00001)
gamma = 1.0
sigma = 0.005
dt = 1/264
r_ = r0 # Este dato no nos fue dado pero supondremos que es igual a r0

dt_gamma_r_ = dt * gamma * r_
sigma_sqdt = sigma * math.sqrt(dt)
gamma_dt = gamma * dt

r = r0
r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * 0.5504

print(f'El valor de r a un paso sería: {r:.8%}')

El valor de r a un paso sería: 0.09755847%


### Pregunta 2

$$TNA=Round\left(\left( \frac{ICP_{T}}{ICP_{t}}-1\right)\cdot\frac{360}{d},4\right)$$

Donde:

- TNA: significa *tasa nominal anual*.
- $ICP_{\tau}$: es el valor del $ICP$ la fecha $\tau=t,T$.
- $d$ es el número de días entre $t$ y $T$.
- $Round\left(x,4\right)$ redondea el número $x$ a 4 decimales.

In [34]:
ICP0 = 10000
ICP365 = 10110

TNA = round((ICP365/ICP0 - 1)*360/365, 4)
print(f'El valor de la TNA sería: {TNA:.2%}')

El valor de la TNA sería: 1.08%


### Pregunta 3

Considere un swap a 1Y con cupones semestrales de 1.2% (lineal). Si la tasa
cupón cero a 0.5Y es 1.1% (exp), calcule la tasa cero a 1Y.

In [57]:
# Swap a 1Y tasa 1.2% lineal
# Z(0.5) = 1.1% (exp)
# Z(1) = ?

x = math.exp(-0.011*0.5) * 0.012 * 0.5 

P = 1 - x / (1 + 0.012*1)

# P = exp(-rt)
# log(P) = -rt
# r = log(P)/-t

rzero = np.log(P)/-1

print(f'El valor de la tasa cero a 1Y es: {rzero:.8%}')


El valor de la tasa cero a 1Y es: 0.59137866%


### Pregunta 4

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.

#### Respuesta:
La diferencia reside en el drift del modelo (dinámicas de r). Si recordamos el modelo de Hull-White, las dinámicas de r son:
$$ dr_t=\left(\theta_t-\gamma^* r_t\right)dt+\sigma dX_t $$
Siendo $\theta_t$ una función determinística del tiempo. En la nueva medida T-forward, la expresión de $\theta_t$ cambia por $\bar{\theta_t}$:
$$
\overline{\theta_t}=\theta_t-B\left(t,T\right)\sigma^2
$$
Siendo $B(t,T)$:
$$
B\left(t,T\right)=\frac{1}{\gamma^*}\left[1-\exp\left(-\gamma^* (T-t)\right)\right)]
$$

Y como cambia la valorización? la valorización cambia simplemente en la forma de descontar los Payoffs. En la medida libre de riesgo debemos devolvernos por la curva multiplicando todos los df de cada paso temporal, sin embargo, con la medida T-forward usamos el factor de descuento asociado a la curva zero cupón en el plazo del payoff T.

### Pregunta 5

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 99.0%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.

In [38]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


#### Respuesta

In [44]:
k = 0.99
Call = hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma)

print(f'El valor de la Call es: {Call:.8}')

El valor de la Call es: 0.0095605971


### Pregunta 6

In [52]:
# Pregunta:
# Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
# forward entre 4282 y 5454 días.

t1 = 4282
t2 = 5454

r1 = curva(t1)
r2 = curva(t2)

df1 = math.exp(-r1*t1/365)
df2 = math.exp(-r2*t2/365)

fwd = (df1/df2 - 1) * 360 / (t2 - t1)


print(f'El valor de la tasa forward es: {fwd:.8%}')

El valor de la tasa forward es: 1.27726337%


### Pregunta 7

Es falso, ya que el modelo de Hull-White es una versión extendida del modelo de Vasicek, en la cual el drift "transforma" su estructura de los parámetros para dar nacimiento a la función $\theta_t$. Como se comentó antes, es una función determinística del tiempo, la cual permite ajustar perfectamente el modelo al valor de mercado de los bonos cupón cero ($Z$). Dada esta diferencia, el valor de $Z$ siempre será levemente distinto, con lo cual el valor de una opción sobre $Z$ también lo será.